# Challenge Data Engineer

Este Notebook contiene las soluciones para resolver tres problemas usando la libreria de pyspark, consta de 4 secciones:
- Inicialización, en donde se importa las librerias, constantes y funciones que se usan en el resto del código
- Una sección por cada uno de los tres problemas del reto. Cada solución contiene la descripción, las soluciones y los resultados

## Inicialización

Ejecutar esta linea de codigo sin comentar (borrar #) solamente una vez al inicio de la sesión para instalar el package de emoji

In [1]:
#!pip install emoji

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, to_date, col, desc, row_number
from pyspark.sql.types import ArrayType, StringType
import pyspark.sql.functions as F

from pyspark.sql.window import Window

from typing import List, Tuple
import datetime

### Definición de constantes

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

Definir una función que genera la session de Spark u obtiene una versión ya existente

In [3]:
def get_spark_session() -> SparkSession:
    return (SparkSession.builder
            .appName("TwitterAnalysis")
            .getOrCreate())

## Problema 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene
por cada uno de esos días. Debe incluir las siguientes funciones:
- def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
- def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:

### Solución q1_memory

In [10]:

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    spark = get_spark_session()

    date_format = "yyyy-MM-dd'T'HH:mm:ssXXX"
    
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("date", "user.username")
                       .withColumn("date", to_date(col("date"), date_format))
               )

    top_dates_df = (tweetsDF.groupBy("date")
                           .agg(count("*").alias("count"))
                           .orderBy(desc("count"))
                           .limit(10)
                   )


    top_dates_set = {row['date'] for row in top_dates_df.collect()}
    filtered_df = tweetsDF.filter(col("date").isin(top_dates_set))

    grouped_df = (filtered_df.groupBy("date", "username")
                             .agg(count("*").alias("count"))
                  )

    windowSpec = Window.partitionBy("date").orderBy(desc("count"))
    
    top_user_df = (grouped_df.withColumn("row_number", row_number().over(windowSpec))
                          .filter(col("row_number") == 1)
                          .drop("row_number")
                  )
    top_user_df = top_user_df.withColumnRenamed("count", "tweets_by_user")
    
    top_dates_with_users = (top_user_df.join(top_dates_df, "date")
                                   .orderBy(desc("count"), "date")
                           )

    return [(row['date'], row['username']) for row in top_dates_with_users.collect()]

Presentación de los resultados

In [11]:
print("----- memory -------")
q1_general_result = q1_general(file_path)
for e in q1_general_result:
    print(e)

----- general -------
(datetime.date(2021, 2, 12), 'RanbirS00614606')
(datetime.date(2021, 2, 13), 'MaanDee08215437')
(datetime.date(2021, 2, 17), 'RaaJVinderkaur')
(datetime.date(2021, 2, 16), 'jot__b')
(datetime.date(2021, 2, 14), 'rebelpacifist')
(datetime.date(2021, 2, 18), 'neetuanjle_nitu')
(datetime.date(2021, 2, 15), 'jot__b')
(datetime.date(2021, 2, 20), 'MangalJ23056160')
(datetime.date(2021, 2, 23), 'Surrypuria')
(datetime.date(2021, 2, 19), 'Preetm91')


### Solución q1_time

In [4]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    spark = get_spark_session()

    date_format = "yyyy-MM-dd'T'HH:mm:ssXXX"
    
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("date", "user.username")
                       .withColumn("date", to_date(col("date"), date_format))
               ).cache()  # Cache tweetsDF as it is used in multiple operations

    top_dates_df = (tweetsDF.groupBy("date")
                           .agg(count("*").alias("count"))
                           .orderBy(desc("count"))
                           .limit(10)
                   ).cache()  # Cache top_dates_df as it is used in multiple operations

    # Use broadcast join for smaller DataFrame to speed up join operation
    filtered_df = tweetsDF.join(F.broadcast(top_dates_df), "date")

    grouped_df = (filtered_df.groupBy("date", "username")
                             .agg(count("*").alias("count"))
                  ).cache()  # Cache grouped_df as it is used in multiple operations

    windowSpec = Window.partitionBy("date").orderBy(desc("count"))
    
    top_user_df = (grouped_df.withColumn("row_number", row_number().over(windowSpec))
                          .filter(col("row_number") == 1)
                          .drop("row_number")
                  ).withColumnRenamed("count", "tweets_by_user")
    
    top_dates_with_users = (top_user_df.join(top_dates_df, "date")
                                   .orderBy(desc("count"), "date")
                           )

    return [(row['date'], row['username']) for row in top_dates_with_users.collect()]

In [5]:
print("----- time -------")
q1_time_result = q1_time(file_path)
for e in q1_time_result:
    print(e)

----- general -------
(datetime.date(2021, 2, 12), 'RanbirS00614606')
(datetime.date(2021, 2, 13), 'MaanDee08215437')
(datetime.date(2021, 2, 17), 'RaaJVinderkaur')
(datetime.date(2021, 2, 16), 'jot__b')
(datetime.date(2021, 2, 14), 'rebelpacifist')
(datetime.date(2021, 2, 18), 'neetuanjle_nitu')
(datetime.date(2021, 2, 15), 'jot__b')
(datetime.date(2021, 2, 20), 'MangalJ23056160')
(datetime.date(2021, 2, 23), 'Surrypuria')
(datetime.date(2021, 2, 19), 'Preetm91')
